In [1]:
import pvdeg
from pvdeg.geospatial import output_template
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

import pandas as pd
import xarray as xr
import dask.array as da
import pickle

from dask.distributed import LocalCluster, Client
from dask_jobqueue import SLURMCluster

In [2]:
cluster = SLURMCluster(
    queue='shared',
    account="pvsoiling",
    #cores=6,
    cores=2,
    #memory="80GB,
    memory="40 GB",
    processes=True,
    log_directory='/home/tford/dev/dask-logs',
    walltime="02:00:00",  # Request a longer time
    scheduler_options={
        "dashboard_address": ":12345"
    }
)
cluster.scale(2)

client = Client(cluster)

print(client.dashboard_link)

http://10.60.1.202:12345/status


In [ ]:
workers = 8

cluster = LocalCluster(
    n_workers=workers,
    processes=True, 
)

client = Client(cluster)

print(client.dashboard_link)

In [ ]:
locationGetter = pvdeg.scenario.GeospatialScenario()

locationGetter.addLocation(country="United States", downsample_factor=9, nsrdb_attributes=pvdeg.pysam.INSPIRE_NSRDB_ATTRIBUTES)

In [ ]:
locationGetter.plot_coords()

In [ ]:
locationGetter.meta_data.iloc[0].to_dict(),

In [ ]:
# this shows us that the data for poa_front is repeated 25 times for tmy, all years are the samer
conf = "01"

res = pvdeg.pysam.pysam(
    pvdeg.weather.roll_tmy(locationGetter.weather_data.isel(gid=100).to_dataframe(), locationGetter.meta_data.iloc[100].to_dict()),
    locationGetter.meta_data.iloc[100].to_dict(),
    pv_model = "pysamv1",
    config_files = {"pv" : f'/home/tford/dev/InSPIRE-1yr/Studies/USMap_Doubleday_2024/SAM/{conf}/{conf}_pvsamv1.json'}
)

# for i in range(25):
#     for j in range(25):
#         if i == j:
#             print("same year")
#             continue

#         if res["poa_rear"][j * 8760 : (j * 8760) + 1] == res["poa_rear"][i * 8760 : i * 8760 + 1]:
#             print("equal")    
#         else:
#             print(i, j, "not equal")

In [ ]:
locationGetter.weather_data.isel(gid=100).dni.plot()

In [ ]:
locationGetter.meta_data.iloc[100].to_dict()

In [ ]:
res.keys()

In [ ]:
res["subarray1_idealrot"]

In [ ]:
inspire_ds = pvdeg.pysam.inspire_ground_irradiance(
    locationGetter.weather_data.isel(gid=0).to_dataframe(),  # function handles UTC -> local for us
    locationGetter.meta_data.iloc[0].to_dict(),
    config_files = {"pv" : '/home/tford/dev/InSPIRE/Studies/USMap_Doubleday_2024/SAM/01/01_pvsamv1.json'}
)   

In [ ]:
inspire_ds.compute()

In [ ]:
geo_weather, geo_meta = locationGetter.get_geospatial_data()

geo_meta = geo_meta[geo_meta['state'] != "Alaska"]
geo_weather = geo_weather.sel(gid=geo_meta.index).chunk({"gid":4}).isel(gid=slice(0, 1600)) # exclusive upper bound

In [ ]:
shapes = {
    "annual_poa": ("gid",),
    "annual_energy": ("gid",),
    "poa_front": ("gid", "time"),
    "poa_rear": ("gid", "time"),
    "subarray1_poa_front": ("gid", "time"),
    "subarray1_poa_rear": ("gid", "time"),
    "temp_air": ("gid", "time"),
    "wind_speed": ("gid", "time"),
    "wind_direction": ("gid", "time"),
    "dhi": ("gid", "time"),
    "ghi": ("gid", "time"),
    "dni": ("gid", "time"),
    "relative_humidity": ("gid", "time"),
    "albedo": ("gid", "time"),
    "ground_irradiance": ("gid", "time", "distance")
}

template = pvdeg.geospatial.output_template(
    ds_gids = geo_weather.isel(gid=slice(500,510)), # times will cause error, fix below
    shapes = shapes,
    add_dims={"distance" : 10}, # this will autogenerate a range of length 10 for the coordinate axis
)

# modified range as produced by the corrected times for the tmy dataset
# might just be able to set this to geo weather time because function returns UTC
#template['time'] = pd.date_range(start='2001-01-01 00:30:00', freq='1h', periods=8760)

template

In [ ]:
client.scatter(geo_meta, broadcast=True)
# we do not want to scatter weather because it is larger than memory and will cause problems

In [ ]:
confs = ["01", "02", "03"]

for conf in confs:

    pysam_tiny_res = pvdeg.geospatial.analysis(
        weather_ds = geo_weather.isel(gid=slice(500,510)).chunk({"gid":1}),
        meta_df = geo_meta.iloc[500:510],
        func = pvdeg.pysam.inspire_ground_irradiance, 
        template = template.chunk({"gid":1}),
        
        config_files = {"pv" : f'/home/tford/dev/InSPIRE/Studies/USMap_Doubleday_2024/SAM/{conf}/{conf}_pvsamv1.json'}
    )
    
    pysam_tiny_res.to_netcdf(f"agrivoltaics-irradiance-conf{conf}.h5", engine="h5netcdf")

In [ ]:
import os

os.getcwd()

In [ ]:
loaded = xr.open_dataset('downsample20-conf01-pysam-ground-irradiance.nc', engine="h5netcdf").compute()

In [ ]:
pysam_tiny_res